In [7]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import shutil
from shutil import copyfile


In [8]:
# Namelist

year=2018 # year of runoff file

# PATHS (Careful !!! Full path needed (e.g: /DATA/ROBERT/ instead of DATA/ROBERT))
rnf_bdy_folder='/data/vdi/tbrivoal/PRE_PROCESSING_IMMERSE/RUNOFFS_eNEATL36/rnf_forcing/'
CLIM_rnf_2D_folder='/data/vdi/tbrivoal/PRE_PROCESSING_IMMERSE/RUNOFFS_eNEATL36/rnf_forcing_daily/'
coords_bdy_file='/data/vdi/tbrivoal/PRE_PROCESSING_IMMERSE/RUNOFFS_eNEATL36/coordinates.bdy.nc'
domain_cfg_file = '/data/vdi/tbrivoal/PRE_PROCESSING_IMMERSE/RUNOFFS_eNEATL36/DOMAIN_CFG_FINALDOMAIN/domain_cfg_init.nc'

output_folder="/data/vdi/tbrivoal/PRE_PROCESSING_IMMERSE/RUNOFFS_eNEATL36/"

test_script=False # For debugging

In [9]:
############# Read data ###############
outfile = output_folder + "runoff_eNEATL36_rivers_y" + str(year) + '.nc'  
#file_rnf_2D=xr.open_dataset(CLIM_rnf_2D_folder + "eNEATL36_1d_runoffs_y" + str(year) + ".nc")
file_rnf_2D=xr.open_dataset(CLIM_rnf_2D_folder + "eNEATL36_1d_runoffs_y2017.nc")


file_rnf_bdy_U=xr.open_dataset(rnf_bdy_folder + "runoff_U_y" + str(year) + ".nc") # file format : runoff_U_yYYYY.nc
file_rnf_bdy_V=xr.open_dataset(rnf_bdy_folder + "runoff_V_y" + str(year) + ".nc")
file_coords_bdy=xr.open_dataset(coords_bdy_file)
domain_cfg = xr.open_dataset(domain_cfg_file ,drop_variables={"x", "y",})

# Read river mouth coordinates
# ghosts cells are taken into account so we have to do - 2 (- 1 for ghost an -1 for python)
time_counter=file_rnf_2D.time_counter.squeeze()
nbiu_gridT = file_rnf_bdy_U.nbidta.squeeze() #- 1 # - 1 to convert to gridT
nbju_gridT  = file_rnf_bdy_U.nbjdta.squeeze()  -1 # - 2 
nbiv_gridT  = file_rnf_bdy_V.nbidta.squeeze()  - 1 # -2 
nbjv_gridT  = file_rnf_bdy_V.nbjdta.squeeze() # - 1 # - 1 to convert to gridT

# Read BDY files of U & V runoffs

U_rnf_bdy = abs(file_rnf_bdy_U.runoffu.squeeze()) #.sel(time_counter=time_counter) 
V_rnf_bdy = abs(file_rnf_bdy_V.runoffv.squeeze()) #.sel(time_counter=time_counter) 

# Read 2D runoff data
rnf_2D = file_rnf_2D.runoffs_instant.squeeze()

# Read coordinates

mask =  domain_cfg.top_level.squeeze()
e1t =  domain_cfg.e1t.squeeze()
e2t =  domain_cfg.e2t.squeeze()


In [10]:
print(rnf_2D.shape)
U_rnf_bdy.shape

(365, 1894, 1294)


(365, 24)

In [11]:
############################ Append 3D bdy runoffs to the climatology ###################################

Rho = 1000.

# now add runoff BDY data over the climatology data
if test_script==True:
    rnf_2D_new = rnf_2D # initialisation
    rnf_2D_new[:,:,:] = 0. # initialisation
else:
    rnf_2D_new = rnf_2D # initialisation
    
rnf_2D_new[:,:,:] = 0.    
for ind in range(len(U_rnf_bdy[0,:])):
#     print(ind, nbju_gridT[ind].values,nbiu_gridT[ind].values)
    e1te2t=e1t[nbju_gridT[ind],nbiu_gridT[ind]] * e2t[nbju_gridT[ind],nbiu_gridT[ind]]
    rnf_2D_new[:,nbju_gridT[ind],nbiu_gridT[ind]] = \
    rnf_2D_new[:,nbju_gridT[ind],nbiu_gridT[ind]]   \
    + ((U_rnf_bdy[:,ind].values * Rho ) / e1te2t.values)
    
    
for ind in range(len(V_rnf_bdy[0,:])):
    e1te2t=e1t[nbjv_gridT[ind],nbiv_gridT[ind]] * e2t[nbjv_gridT[ind],nbiv_gridT[ind]]
    rnf_2D_new[:,nbjv_gridT[ind],nbiv_gridT[ind]] = \
    rnf_2D_new[:,nbjv_gridT[ind],nbiv_gridT[ind]] \
    + ((V_rnf_bdy[:,ind].values * Rho ) / e1te2t.values)

In [12]:
#
if test_script==True:
    TEST = rnf_2D_new.where(mask.values==0,0.) # mask sea data for testing
    if np.nanmean(TEST.values) == 0:
        print("TEST OK")
    else:
        print("TEST NOK", str(np.nanmean(TEST.values)))


In [13]:
rnf_2D_new

<xarray.DataArray 'runoffs_instant' (time_counter: 365, y: 1894, x: 1294)>
[894555140 values with dtype=float32]
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2017-01-01T12:00:00 ... 2017-...
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
Dimensions without coordinates: y, x
Attributes:
    standard_name:       water_flux_into_sea_water_from_rivers
    long_name:           River Runoffs
    units:               kg/m2/s
    online_operation:    instant
    interval_operation:  150 s
    interval_write:      1 d
    cell_methods:        time: point (interval: 150 s)

In [14]:
############################ Save in a Netcdf ###################################

# time_counter_daily_float = pd.to_numeric(time_counter_daily, downcast='float')
#time_counter_daily.values.astype("float")
# Create xarray dataarray
rnf_2D_new_da = xr.DataArray(
    data=rnf_2D_new,
    dims=["time_counter", "y", "x"],
    coords=rnf_2D.coords,
    attrs=rnf_2D.attrs,
    name=rnf_2D.name,
     )



# # Create dataset
# ds = xr.merge([rnf_2D_new_da, file_rnf_2D.nav_lat, file_rnf_2D.nav_lon])
# ds.attrs=file_rnf_2D.attrs
# # Save to netcdf
rnf_2D_new_da.to_netcdf(outfile, mode="w")

